<a href="https://colab.research.google.com/github/KunjShah95/FINE-TUNED-MODELS/blob/main/finetuned_GEMMA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
!pip install -qqq git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3 \
                  git+https://github.com/huggingface/trl.git@main \
                  bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText
from peft import LoraConfig, get_peft_model
model = AutoModelForImageTextToText.from_pretrained(
    "google/gemma-3-4b-it", device_map="auto", torch_dtype=torch.bfloat16, attn_implementation="eager"
)
lora_config = LoraConfig(task_type="CAUSAL_LM", r=16, lora_alpha=32, target_modules="all-linear")
model = get_peft_model(model, lora_config)
processor = AutoProcessor.from_pretrained("google/gemma-3-4b-it")
tokenizer = processor.tokenizer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [ ]:
reasoning = """<reasoning>
  <step>Speed = Distance / Time</step>
  <step>60 km / 3 hours = 20 km/h</step>
  <step>Distance in 5 hours = 20 km/h * 5 hours = 100 km</step>
</reasoning>"""

answer = "<answer>100 km</answer>"

print(reasoning)
print(answer)

<reasoning>
  <step>Speed = Distance / Time</step>
  <step>60 km / 3 hours = 20 km/h</step>
  <step>Distance in 5 hours = 20 km/h * 5 hours = 100 km</step>
</reasoning>
<answer>100 km</answer>


In [ ]:
from datasets import load_dataset
SYSTEM_PROMPT = "Respond in structured reasoning format (XML)."
def get_gsm8k_questions(split="train"):
    data = load_dataset('openai/gsm8k', 'main')[split]
    return data.map(lambda x: {
        'prompt': [{'role': 'system', 'content': SYSTEM_PROMPT}, {'role': 'user', 'content': x['question']}],
        'answer': x['answer']
    })
train_data = get_gsm8k_questions()


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

In [ ]:
def correctness_reward_func(prompts, completions, answer):
    responses = [extract_xml_answer(c[0]['content']) for c in completions]
    return [2.0 if r == a else 0.0 for r, a in zip(responses, answer)]


In [ ]:
merged_model = model.merge_and_unload()
merged_model.push_to_hub("kunjshah01/gemma-reasoning-genius")


model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kunjshah01/gemma-reasoning-genius/commit/5b008ada67f28aab60b49a1860f4720e5bca2ac0', commit_message='Upload Gemma3ForConditionalGeneration', commit_description='', oid='5b008ada67f28aab60b49a1860f4720e5bca2ac0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kunjshah01/gemma-reasoning-genius', endpoint='https://huggingface.co', repo_type='model', repo_id='kunjshah01/gemma-reasoning-genius'), pr_revision=None, pr_num=None)